##  Pump it Up: Data Mining the Water Table

https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/page/23/




**Importing Libraries**

In [1]:
import numpy as np #for linear algebra
import pandas as pd #for data processing

#for data visualization
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import style

#we would use models from the scikit learn library to develop our machine learning models

#Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.metrics import roc_curve, auc

#Models
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB

import string

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

**Reading in Data**

In [23]:
X_train = pd.read_csv('/content/Trainig set values.csv').set_index('id')
y_train = pd.read_csv('/content/Training set labels.csv').set_index('id')

X_test = pd.read_csv('/content/Test set values.csv').set_index('id')
submission = pd.read_csv('/content/SubmissionFormat.csv').set_index('id')

**Exploratory Data Analysis (EDA)**

**Description of the Features**

amount_tsh - Total static head (amount water available to waterpoint) (Not sure)

date_recorded - The date the row was entered (Not sure)

funder - Who funded the well. This is a categorical feature with 1897 unique values (Might affect target variable)

gps_height - Altitude of the well (Might affect target variable)

installer - Organization that installed the well, This is a categorical feature with 2145 unique values (might affect target variable)

longitude - GPS coordinate (might affect)
latitude - GPS coordinate (might affect)

wpt_name - Name of the waterpoint if there is one (might not affect)
num_private -

basin - Geographic water basin, his is a categorical feature with 9 unique values (might affect)

subvillage - Geographic location (might affect)

region - Geographic location (might affect)

region_code - Geographic location (coded) (might not affect)

district_code - Geographic location (coded) (might not affect)

lga - Geographic location (might affect)

ward - Geographic location (might affect)

population - Population around the well (might affect)

public_meeting - True/False (might affect)

recorded_by - Group entering this row of data (might not affect)

scheme_management - Who operates the waterpoint (might affect)

scheme_name - Who operates the waterpoint (might not affect)

permit - If the waterpoint is permitted (might affect)

construction_year - Year the waterpoint was constructed (would affect)

extraction_type - The kind of extraction the waterpoint uses (might affect)

extraction_type_group - The kind of extraction the waterpoint uses (might not affect)

extraction_type_class - The kind of extraction the waterpoint uses (might not affect)

management - How the waterpoint is managed (would affect)

management_group - How the waterpoint is managed (would affect)

payment - What the water costs (might affect)

payment_type - What the water costs (might affect)

water_quality - The quality of the water (might affect)

quality_group - The quality of the water (might affect)

quantity - The quantity of water (might affect)

quantity_group - The quantity of water (might affect)

source - The source of the water (might affect)

source_type - The source of the water (might affect)

source_class - The source of the water (might affect)

waterpoint_type - The kind of waterpoint (might affect)

waterpoint_type_group - The kind of waterpoint (might affect)

In [20]:
X_train.head()

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [70]:
X_train['date_recorded'] = pd.to_datetime(X_train['date_recorded'])

In [71]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 69572 to 26348
Data columns (total 39 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   amount_tsh             59400 non-null  float64       
 1   date_recorded          59400 non-null  datetime64[ns]
 2   funder                 55765 non-null  object        
 3   gps_height             59400 non-null  int64         
 4   installer              55745 non-null  object        
 5   longitude              59400 non-null  float64       
 6   latitude               59400 non-null  float64       
 7   wpt_name               59400 non-null  object        
 8   num_private            59400 non-null  int64         
 9   basin                  59400 non-null  object        
 10  subvillage             59029 non-null  object        
 11  region                 59400 non-null  object        
 12  region_code            59400 non-null  int64         
 1

Our training set has 59,400 entries and 38 features, this does not include the target variable or the label (status_group). They are datetime64[ns](1) that is 1 datetime feature, float64(3) three floats, int64(6), six integers and twenty nine objects, object(29).

The function below is used to identify unique values of our categorical variables

In [73]:
def unique_values(column):
    unique = X_train[column].unique()
    no_of_unique_values = X_train[column].nunique()
    print('Number of unique values is ' + str(no_of_unique_values))
    return unique
    

In [74]:
unique_values('waterpoint_type')

Number of unique values is 7


array(['communal standpipe', 'communal standpipe multiple', 'hand pump',
       'other', 'improved spring', 'cattle trough', 'dam'], dtype=object)

**Percentage of missing values**

In [13]:
#So we define a function for this purpose
def missing_values_tables(df):

  #find sum of missing values
  missing_values = df.isnull().sum()

  #find %age of missing values
  missing_values_percent = 100 * df.isnull().sum()/len(df)

  #make a table with the results
  mis_val_table = pd.concat( [missing_values, missing_values_percent], axis=1)

  #rename the columns
  mis_val_table_ren_columns = mis_val_table.rename(columns={0: 'Missing values', 1 : '% of Total Values'})

  #sort the table by percentage of missing data in descending order
  mis_val_table_ren_columns =  mis_val_table_ren_columns[mis_val_table_ren_columns.iloc[:,1] !=0].sort_values(
      '% of Total Values', ascending= False).round(1)

  #print some summary info
  print('Your selected dataframe has ' + str(df.shape[1]) + ' columns.\n '
        'There are ' + str(mis_val_table_ren_columns.shape[0])  +  ' columns that have missing values')
  
  #return the dataframe with missing info
  return mis_val_table_ren_columns

In [24]:
missing_values_tables(X_train)

Your selected dataframe has 39 columns.
 There are 7 columns that have missing values


,Missing values,% of Total Values
scheme_name,28166,47.4
scheme_management,3877,6.5
installer,3655,6.2
funder,3635,6.1
public_meeting,3334,5.6
permit,3056,5.1
subvillage,371,0.6


In [12]:
X_train.describe()

,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,15496.000000,15496.000000,15496.000000,1.549600e+04,15496.000000,15496.000000,15496.000000,15496.000000,15496.000000
mean,343.155034,665.722315,34.157293,-5.725727e+00,0.390811,15.515617,5.721606,179.313178,1302.728898
std,3872.741926,692.568185,6.429530,2.943184e+00,7.501263,17.936974,9.792553,461.985503,950.934484
min,0.000000,-63.000000,0.000000,-1.158630e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,33.111923,-8.572355e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,0.000000,365.000000,34.915983,-5.026668e+00,0.000000,12.000000,3.000000,25.000000,1985.000000
75%,25.000000,1317.000000,37.206410,-3.327171e+00,0.000000,17.000000,5.000000,215.000000,2004.000000
max,350000.000000,2627.000000,40.345193,-2.000000e-08,698.000000,99.000000,80.000000,15300.000000,2013.000000


In [15]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14850 entries, 50785 to 68707
Data columns (total 39 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   amount_tsh             14850 non-null  float64
 1   date_recorded          14850 non-null  object 
 2   funder                 13981 non-null  object 
 3   gps_height             14850 non-null  int64  
 4   installer              13973 non-null  object 
 5   longitude              14850 non-null  float64
 6   latitude               14850 non-null  float64
 7   wpt_name               14850 non-null  object 
 8   num_private            14850 non-null  int64  
 9   basin                  14850 non-null  object 
 10  subvillage             14751 non-null  object 
 11  region                 14850 non-null  object 
 12  region_code            14850 non-null  int64  
 13  district_code          14850 non-null  int64  
 14  lga                    14850 non-null  object 
 15

In [16]:
y_train.head()

,status_group
id,
69572,functional
8776,functional
34310,functional
67743,non functional
19728,functional


In [17]:
y_train['status_group'].value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

In [18]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 69572 to 26348
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   status_group  59400 non-null  object
dtypes: object(1)
memory usage: 928.1+ KB


Some Plots

In [25]:
import folium

In [26]:
center = X_train[['latitude', 'longitude']].mean()

center

latitude     -5.706033
longitude    34.077427
dtype: float64

In [27]:
colors = {'functional': 'blue', 'non functional': 'gray', 'functional needs repair': 'red'}

m = folium.Map(location=center, zoom_start=6)

for pump in X_train.sample(200).itertuples():
    #folium.Marker(location=[pump.latitude, pump.longitude], 
    #             icon=folium.Icon(color=colors[y_train.loc[pump.Index, 'status_group']], icon='none')).add_to(m)
    folium.CircleMarker(location=[pump.latitude, pump.longitude], radius = 3, 
                        color=colors[y_train.loc[pump.Index, 'status_group']]).add_to(m)
m

**Model Building**

In [30]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [40]:
pipe = Pipeline([
    ('features', ColumnTransformer([
        ('keep', 'passthrough', ['amount_tsh', 'latitude', 'longitude', 'construction_year','population' ])
    ])),
    ('classifier', GridSearchCV(RandomForestClassifier(n_estimators=50, random_state=117),
                               param_grid={'max_depth': [10, 20, 30, 40]}, cv=5))
])

In [41]:
pipe.fit(X_train, y_train.values.reshape(-1))

Pipeline(memory=None,
         steps=[('features',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('keep', 'passthrough',
                                                  ['amount_tsh', 'latitude',
                                                   'longitude',
                                                   'construction_year',
                                                   'population'])],
                                   verbose=False)),
                ('classifier',
                 GridSearchCV(cv=5, error_score=nan,
                              estimator=RandomForestClassifier(bootstrap=T...
                                                               min_impurity_decrease=0.0,
                                                               min_impurity_split=None,
                   

In [42]:
pipe.score(X_train, y_train.values.reshape(-1))

0.9347474747474748

In [43]:
pipe.named_steps['classifier'].best_params_

{'max_depth': 20}

In [44]:
#To make our prediction
submission['status_group'] = pipe.predict(X_test)

In [45]:
#To export our submission
date_today = pd.datetime.now().strftime("%m-%d-%Y-%H:%M")
submission.to_csv(f'submission-{date_today}.csv')

Below is another way to export your data

In [ ]:
#first_submission = pd.DataFrame(pipe.predict(X_test), index = X_test.index, columns = ['status_group'])

In [ ]:
#first_submission.head()

In [ ]:
#from google.colab import files
#dd_submission1 = first_submission.to_csv('ddsubmission.csv')
#files.download('ddsubmission.csv')